# Vectara

[Vectara](https://vectara.com/) 是一个值得信赖的 AI 助手和代理平台，专注于为关键任务应用提供企业级支持。
Vectara 无服务器 RAG 即服务提供 RAG 的所有组件，并通过易于使用的 API 提供，包括：
1. 从文件（PDF、PPT、DOCX 等）中提取文本的方法
2. 提供最先进性能的基于 ML 的分块。
3. [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) 嵌入模型。
4. 其内部向量数据库，用于存储文本分块和嵌入向量。
5. 一个查询服务，可自动将查询编码为嵌入，并检索最相关的文本片段，支持 [混合搜索](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching) 以及多种重新排序选项，如 [多语言相关性重新排序器](https://www.vectara.com/blog/deep-dive-into-vectara-multilingual-reranker-v1-state-of-the-art-reranker-across-100-languages)、[MMR](https://vectara.com/get-diverse-results-and-comprehensive-summaries-with-vectaras-mmr-reranker/)、[UDF 重新排序器](https://www.vectara.com/blog/rag-with-user-defined-functions-based-reranking)。
6. 一个 LLM，用于根据检索到的文档（上下文）创建 [生成式摘要](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview)，包含引文。

更多信息：
- [文档](https://docs.vectara.com/docs/)
- [API Playground](https://docs.vectara.com/docs/rest-api/)
- [快速入门](https://docs.vectara.com/docs/quickstart)

本笔记本展示了在仅将 Vectara 用作向量存储（无摘要）的情况下，如何使用其基本检索功能，包括：`similarity_search` 和 `similarity_search_with_score`，以及使用 LangChain 的 `as_retriever` 功能。


## 设置

要使用 `VectaraVectorStore`，首先需要安装配套包。

In [ ]:
!uv pip install -U pip && uv pip install -qU langchain-vectara

# 入门

开始之前，请按照以下步骤操作：
1. 如果您还没有账户，请[注册](https://www.vectara.com/integrations/langchain)免费试用 Vectara。
2. 在您的账户中，您可以创建一个或多个语料库。每个语料库代表一个区域，用于在摄入输入文档时存储文本数据。要创建语料库，请使用“**创建语料库**”按钮。然后，为您要创建的语料库提供名称和描述。您可以选择定义过滤属性并应用一些高级选项。如果单击您创建的语料库，您可以在顶部看到其名称和语料库 ID。
3. 接下来，您需要创建 API 密钥才能访问语料库。在语料库视图中单击“**访问控制**”选项卡，然后单击“**创建 API 密钥**”按钮。为您的密钥命名，并选择是希望密钥仅用于查询还是用于查询+索引。单击“创建”即可获得一个有效的 API 密钥。请妥善保管此密钥。

要将 LangChain 与 Vectara 结合使用，您需要以下两个值：`corpus_key` 和 `api_key`。
您可以通过以下两种方式将 `VECTARA_API_KEY` 提供给 LangChain：

1. 在您的环境中包含这两个变量：`VECTARA_API_KEY`。

   例如，您可以使用 os.environ 和 getpass 设置这些变量，如下所示：

```python
import os
import getpass

os.environ["VECTARA_API_KEY"] = getpass.getpass("Vectara API Key:")
```

2. 将它们添加到 `Vectara` vectorstore 构造函数中：

```python
vectara = Vectara(
    vectara_api_key=vectara_api_key
)
```

在此 notebook 中，我们假设它们已在环境中提供。

In [11]:
import os

os.environ["VECTARA_API_KEY"] = "<VECTARA_API_KEY>"
os.environ["VECTARA_CORPUS_KEY"] = "VECTARA_CORPUS_KEY"

from langchain_vectara import Vectara
from langchain_vectara.vectorstores import (
    ChainReranker,
    CorpusConfig,
    CustomerSpecificReranker,
    File,
    GenerationConfig,
    MmrReranker,
    SearchConfig,
    VectaraQueryConfig,
)

vectara = Vectara(vectara_api_key=os.getenv("VECTARA_API_KEY"))

首先，我们将 state-of-the-union 的文本加载到 Vectara。

请注意，我们使用的是 `add_files` 接口，该接口不需要任何本地处理或分块——Vectara 接收文件内容，并执行所有必要的前期处理、分块和将文件嵌入其知识库的操作。

在本例中，使用的是 `.txt` 文件，但对于许多其他[文件类型](https://docs.vectara.com/docs/api-reference/indexing-apis/file-upload/file-upload-filetypes)，方法是相同的。

In [12]:
corpus_key = os.getenv("VECTARA_CORPUS_KEY")
file_obj = File(
    file_path="../document_loaders/example_data/state_of_the_union.txt",
    metadata={"source": "text_file"},
)
vectara.add_files([file_obj], corpus_key)

['state_of_the_union.txt']

## 云图 RAG (检索增强生成)

我们现在创建一个 `VectaraQueryConfig` 对象来控制检索和摘要选项：
* 我们启用摘要，指定我们希望 LLM 选择匹配的 top 7 个块，并用英文回复

使用此配置，让我们创建一个 LangChain `Runnable` 对象，该对象封装了完整的云图 RAG 管道，使用 `as_rag` 方法：

In [13]:
generation_config = GenerationConfig(
    max_used_search_results=7,
    response_language="eng",
    generation_preset_name="vectara-summary-ext-24-05-med-omni",
    enable_factual_consistency_score=True,
)
search_config = SearchConfig(
    corpora=[CorpusConfig(corpus_key=corpus_key)],
    limit=25,
    reranker=ChainReranker(
        rerankers=[
            CustomerSpecificReranker(reranker_id="rnk_272725719", limit=100),
            MmrReranker(diversity_bias=0.2, limit=100),
        ]
    ),
)

config = VectaraQueryConfig(
    search=search_config,
    generation=generation_config,
)

query_str = "what did Biden say?"

rag = vectara.as_rag(config)
rag.invoke(query_str)["answer"]

"President Biden discussed several key issues in his recent statements. He emphasized the importance of keeping schools open and noted that with a high vaccination rate and reduced hospitalizations, most Americans can safely return to normal activities without masks [1]. He addressed the need to hold social media platforms accountable for their impact on children and called for stronger privacy protections and mental health services [2]. Biden also announced measures against Russia, including preventing its central bank from defending the Ruble and targeting Russian oligarchs' assets, as part of efforts to weaken Russia's economy and military [3]. Additionally, he highlighted the importance of protecting women's rights, specifically the right to choose as affirmed in Roe v. Wade [5]. Lastly, he advocated for funding the police with necessary resources and training to ensure community safety [6]."

我们也可以像这样使用流式接口：

In [14]:
output = {}
curr_key = None
for chunk in rag.stream(query_str):
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key == "answer":
            print(chunk[key], end="", flush=True)
        curr_key = key

President Biden emphasized several key points in his statements. He highlighted the importance of keeping schools open and noted that with a high vaccination rate and reduced hospitalizations, most Americans can safely return to normal activities without masks [1]. He addressed the need to hold social media platforms accountable for their impact on children and called for stronger privacy protections and mental health services [2]. Biden also discussed measures against Russia, including preventing their central bank from defending the Ruble and targeting Russian oligarchs' assets [3]. Additionally, he reaffirmed the commitment to protect women's rights, particularly the right to choose as affirmed in Roe v. Wade [5]. Lastly, he advocated for funding the police to ensure community safety [6].

有关 Vectara 作为 VectorStore 的更多详情，请访问此笔记本：[../vectorstores/vectara.ipynb]。

## Vectara Chat

在使用 LangChain 创建聊天机器人时，大多数情况下都需要集成一个特殊的 `memory` 组件，该组件维护聊天会话的历史记录，然后利用这些历史记录来确保聊天机器人了解对话的上下文。

借助 Vectara Chat，所有这些操作都由 Vectara 在后端自动执行。

In [15]:
generation_config = GenerationConfig(
    max_used_search_results=7,
    response_language="eng",
    generation_preset_name="vectara-summary-ext-24-05-med-omni",
    enable_factual_consistency_score=True,
)
search_config = SearchConfig(
    corpora=[CorpusConfig(corpus_key=corpus_key, limit=25)],
    reranker=MmrReranker(diversity_bias=0.2),
)

config = VectaraQueryConfig(
    search=search_config,
    generation=generation_config,
)


bot = vectara.as_chat(config)

bot.invoke("What did the president say about Ketanji Brown Jackson?")["answer"]

'The president stated that nominating someone to serve on the United States Supreme Court is one of the most serious constitutional responsibilities he has. He nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, describing her as one of the nation’s top legal minds who will continue Justice Breyer’s legacy of excellence [1].'

如需了解更多关于 Vectara chat 的详情，请[前往此笔记本](../chat/vectara.ipynb)。

## Vectara 作为自查询检索器
Vectara 提供智能查询重写选项，通过从自然语言查询自动生成元数据过滤器表达式来提高搜索精度。此功能会分析用户查询，提取相关的元数据过滤器，并重写查询以聚焦于核心信息需求。更多详情请访问 [此 notebook](../retrievers/self_query/vectara_self_query.ipynb)。

## Vectara 工具

Vectara 提供了一些可与 Langchain 一同使用的工具。更多详情请[访问此笔记本](../tools/vectara.ipynb)。